In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/STECH

/content/drive/MyDrive/STECH


In [3]:
import pandas as pd
import time
import os
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('FX_EURUSD, 240 (16).csv')
len(df)

11477

In [5]:
df = df[df.columns[[0,1,2,3,4,15,17,-2]]]
df.head(10)

,time,open,high,low,close,EMA,MA,Volume
0,2015-01-02T09:00:00+07:00,1.20991,1.20991,1.20460,1.20516,NaN,NaN,9602
1,2015-01-02T13:00:00+07:00,1.20516,1.20696,1.20345,1.20579,NaN,NaN,46744
2,2015-01-02T17:00:00+07:00,1.20579,1.20582,1.20256,1.20308,NaN,NaN,37434
3,2015-01-02T21:00:00+07:00,1.20308,1.20335,1.20037,1.20097,NaN,NaN,52610
4,2015-01-03T01:00:00+07:00,1.20097,1.20155,1.19980,1.19994,NaN,NaN,13246
5,2015-01-05T05:00:00+07:00,1.19551,1.19761,1.18626,1.19546,NaN,NaN,46930
6,2015-01-05T09:00:00+07:00,1.19546,1.19653,1.19259,1.19350,NaN,NaN,26382
7,2015-01-05T13:00:00+07:00,1.19350,1.19761,1.19308,1.19543,NaN,NaN,51356
8,2015-01-05T17:00:00+07:00,1.19543,1.19570,1.18870,1.19130,1.198959,1.198959,62589
9,2015-01-05T21:00:00+07:00,1.19130,1.19414,1.19086,1.19231,1.197629,1.197531,52467


In [6]:
df.drop('time',axis=1,inplace=True)

In [7]:
window_size = 10
result = {}

In [8]:
feature_columns = df.columns.tolist()
feature_columns

['open', 'high', 'low', 'close', 'EMA', 'MA', 'Volume']

In [9]:
column_scaler = {}
for column in feature_columns:
  scaler = preprocessing.MinMaxScaler()
  df[column] = scaler.fit_transform(df[column].values.reshape(-1,1))
  column_scaler[column] = scaler

In [10]:
result['df'] = df.copy()
result['column_scaler'] = column_scaler
df['future'] = df['close'].shift(-window_size)
last_sequence = df[feature_columns].tail(window_size)
result['last_sequence'] = last_sequence

In [11]:
result['df'] = result['df'].iloc[:-window_size,:]
result['df']['future'] = df['future'][:-10]

In [12]:
result['df']['label'] = np.where(result['df']['future'] - result['df']['close'] > 0, 1, 0)

In [13]:
result['df']['label'].value_counts()

0    5833
1    5634
Name: label, dtype: int64

In [14]:
result['last_sequence']

,open,high,low,close,EMA,MA,Volume
11467,0.063119,0.057941,0.057261,0.063119,0.045475,0.053653,0.129166
11468,0.063119,0.074601,0.067430,0.082151,0.049493,0.052744,0.161997
11469,0.082151,0.099314,0.090014,0.107526,0.058028,0.056136,0.229364
11470,0.107526,0.103180,0.110444,0.102186,0.063736,0.060046,0.094591
11471,0.102186,0.094022,0.104535,0.099722,0.067786,0.062936,0.037819
11472,0.099722,0.091307,0.098443,0.098535,0.070777,0.067506,0.068084
11473,0.098535,0.096829,0.098305,0.100224,0.073524,0.073629,0.127317
11474,0.100224,0.099406,0.101237,0.094245,0.074468,0.077645,0.105810
11475,0.094245,0.086612,0.091296,0.087125,0.073730,0.080424,0.162041
11476,0.087125,0.084956,0.088365,0.092282,0.074221,0.083800,0.086775


In [15]:
result['df'].dropna(inplace=True)
result['df'].drop('future',inplace=True,axis=1)
result['df'].reset_index(inplace=True)
result['df'].drop('index', inplace=True, axis=1)

In [16]:
result['df']

,open,high,low,close,EMA,MA,Volume,label
0,0.728629,0.724470,0.708612,0.709780,0.760254,0.759706,0.137929,0
1,0.709780,0.717290,0.718507,0.714390,0.753891,0.752916,0.115592,0
2,0.714390,0.721018,0.721255,0.718634,0.749690,0.746286,0.035155,0
3,0.718634,0.719499,0.726844,0.722468,0.747134,0.741531,0.033730,0
4,0.722468,0.725022,0.733074,0.729862,0.746639,0.738746,0.040657,0
...,...,...,...,...,...,...,...,...
11454,0.068413,0.067329,0.075355,0.074757,0.052897,0.060680,0.109204,1
11455,0.074757,0.073220,0.065231,0.059057,0.050588,0.063137,0.163017,1
11456,0.059057,0.048645,0.054695,0.047328,0.046281,0.061636,0.077306,1
11457,0.047328,0.054121,0.055153,0.059559,0.045400,0.058757,0.052745,1


In [17]:
X = result['df'].iloc[:,0:-1]
y = result['df'].iloc[:,-1]

In [18]:
test_size = 0.2
train_samples = int((1 - test_size)*len(X))

In [19]:
result['X_train'] = X[:train_samples]
result['y_train'] = y[:train_samples]
result['X_test'] = X[train_samples:]
result['y_test'] = y[train_samples:]

In [34]:
lgbm_params = {
    'n_estimators': [100,200,300],
    'learning_rate': [0.01, 0.05,0.1,0.3],
    'num_leaves': [20,50,80,100],
    'max_depth' : [3,4,5,6],
    'num_iterations' : [10,20,30,40,50]
}
seed=0

In [35]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(
    seed=seed,
    objective="binary",
    boosting_type='dart'
)

In [ ]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(
    estimator=lgbm,
    param_grid=lgbm_params,
    cv=5, scoring='accuracy',
    error_score=0
)
grid_result = clf.fit(result['X_train'], result['y_train'])

In [37]:
final_model = lgbm.set_params(**grid_result.best_params_)
final_model.fit(result['X_train'], result['y_train'])

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='dart', learning_rate=0.01, max_depth=5,
               num_iterations=10, num_leaves=20, objective='binary', seed=0)

In [38]:
y_pred = final_model.predict(result['X_test'])

In [39]:
# y_pred

array([0, 0, 0, ..., 1, 0, 0])

In [40]:
# print('Accuracy with LGBM = ',accuracy_score(y_pred, result['y_test']))

Accuracy with LGBM =  0.5292321116928447


In [41]:
SEARCH_PARAMS = {'learning_rate': 0.4,
                 'max_depth': 15,
                 'num_leaves': 20,
                 'feature_fraction': 0.8,
                 'subsample': 0.2}
FIXED_PARAMS={'objective': 'binary',
              'metric': 'auc',
              'boosting':'dart',
              'num_boost_round':300,
              'early_stopping_rounds':30}
params = {'metric':FIXED_PARAMS['metric'],
             'objective':FIXED_PARAMS['objective'],
             **SEARCH_PARAMS}

In [42]:
import lightgbm as lgb
d_train = lgb.Dataset(result['X_train'],label=result['y_train'])
d_test = lgb.Dataset(result['X_test'],label=result['y_test'])
model = lgb.train(params, d_train,                     
                     valid_sets=[d_test],
                     num_boost_round=FIXED_PARAMS['num_boost_round'],
                     early_stopping_rounds=FIXED_PARAMS['early_stopping_rounds'],
                     valid_names=['valid'])
score = model.best_score['valid']['auc']
score

[1]	valid's auc: 0.48804
Training until validation scores don't improve for 30 rounds.
[2]	valid's auc: 0.488419
[3]	valid's auc: 0.491502
[4]	valid's auc: 0.499095
[5]	valid's auc: 0.494454
[6]	valid's auc: 0.494475
[7]	valid's auc: 0.506881
[8]	valid's auc: 0.506864
[9]	valid's auc: 0.509426
[10]	valid's auc: 0.509096
[11]	valid's auc: 0.507674
[12]	valid's auc: 0.508361
[13]	valid's auc: 0.508482
[14]	valid's auc: 0.507708
[15]	valid's auc: 0.51042
[16]	valid's auc: 0.510425
[17]	valid's auc: 0.510741
[18]	valid's auc: 0.510347
[19]	valid's auc: 0.510234
[20]	valid's auc: 0.510391
[21]	valid's auc: 0.510834
[22]	valid's auc: 0.510124
[23]	valid's auc: 0.508917
[24]	valid's auc: 0.5112
[25]	valid's auc: 0.508117
[26]	valid's auc: 0.506554
[27]	valid's auc: 0.506969
[28]	valid's auc: 0.505857
[29]	valid's auc: 0.506283
[30]	valid's auc: 0.506452
[31]	valid's auc: 0.506276
[32]	valid's auc: 0.506926
[33]	valid's auc: 0.507095
[34]	valid's auc: 0.506391
[35]	valid's auc: 0.507155
[36]	v

0.5118729234991884